# Table of Contents
 <p><div class="lev2"><a href="#Lesson2-:-Markov-Decision-Process"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Lesson2 : Markov Decision Process</a></div><div class="lev2"><a href="#Lesson-3-:-Temporal-difference-learning"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Lesson 3 : Temporal difference learning</a></div><div class="lev3"><a href="#TD(1)-learning"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>TD(1) learning</a></div><div class="lev3"><a href="#TD(0)"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>TD(0)</a></div><div class="lev3"><a href="#TD(λ)"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>TD(λ)</a></div>

3つのleaning

* supervised learning $y=f(x)$ function approx.
* unsupervised learning : $f(x)$
* reinforcement learning $y=f(x), z$



Grid world


In [1]:
0.8*0.8*0.9*0.9*0.9

0.4665600000000002

In [3]:
0.8**5+0.1**4*0.8

0.3277600000000001

## Lesson2 : Markov Decision Process
* states :$s$
* model : Model $T(s,a,s') \sim Pr(s'|s,a)$ モデルがひとつ前の状態にしか依存しない→Markov
* actions : $A(s), A$
* reward : $R(s),R(s,a),R(s,a,s')$

solution
* policy $\pi(s) \rightarrow a$

$s,a,r$のペアから$pi$をえる

リワードの設計によって迷路の最適解は変わって学習の結果がかわる


infinite horizons : 寿命を考えると迷路の同じ位置でも状態は変わる

utilityにdiscountをいれないで、ただrewordを足すと正のリワードが得られる場合、無限の時間を考えると、どんなrewordでもutilityは無限で比較できなくなってしまう

best policies

$$
\pi^* := \rm{arg} \max_\pi E[\sum_{t=0}^{\infty} \gamma^t R(s_t) | \pi]
$$

ある状態のutilityは（rewordとは違う)

$$
U^\pi(s) := E[\sum_{t=0}^{\infty} \gamma^t R(s_t)|\pi,s_0=s]
$$

以降では$U(s):=U^{\pi^*}(s)$とする

最適戦略のもとでのつぎの状態s'のutilityの期待値を最大化するようなアクションが最適 : 

$$
\pi^*(s) = \rm{arg}\max_a \sum_{s'} T(s,a,s') U(s')
$$

再帰的に書けば

$$
U(s) = R(s) + \gamma \max_a \sum_{s'} T(s,a,s') U(s')
$$

これはBellman方程式と呼ばれる

Bellman方程式を解きたい。状態数を$n$とすれば、方程式の数は$n$、未知変数は$U(s)$だけなので、$n$個。$\max$によって方程式は非線形になっていて、解析的に解けない。

適当な$\hat{U}_t(s)$から初めて、

$$
\hat{U}_{t+1}(s) = R(s) + \gamma \max_a \sum_{s'} T(s,a,s') \hat{U}_t(s')
$$

と更新する。(value iteration)

なんでいいのかというと、$R(s)$が真のrewardなので、いいっぽい。
なんか局所解にはまりそうだけどなぁ。もっといい$\max$を解く数値計算方法？



理解したいこと:利得がgivenじゃないとき？

迷路の計算例


In [1]:
u_1_x = -0.04 + 0.5 * (0.8*1+0.2*0)
u_1_xd = -0.04
u_2_x = -0.04 + 0.5 *(0.8*1+0.1*u_1_x+0.1*u_1_xd)
u_2_x

0.37600000000000006

別に完璧なUを求める必要はない。policy $\pi$さえわかればいい→policy iteration

まず適当なpolicy $\pi_0$を決める

(1) $\pi_t$があったときに$U_t=U^{\pi_t}$を計算する

$$
U_t(s)= -R(s) + \gamma \sum_{s'} T(s,\pi_t(s), s')U_t(s')
$$

(2) 更新する $\pi_{t+1}=\rm{arg}\max_a \sum T(s,a,s') U(s')$

ポイントは$U$の更新に$\max$がなくなったので、線形になった。

Bellman Equation(より一般に)

$$
V(s) = \max_a(R(s,a)+\gamma \sum_{s'} T(s,a,s') V(s'))
$$

さっきの$R(s)$を$R(s,a)$にして、$U$を$V$にした。$V$をvalue functionと呼ぶ

$$
V(s) = \max_a (R(s,a)+\gamma\sum_{s'} T(s,a,s') (\max_a(R(s,a)+\gamma\sum_{s'} T(s,a,s')\cdots]
$$

なので、この再帰を違うふうに区切ることもできる

$$
Q(s,a) = R(s,a) + \gamma \sum_{s'} T(s,a,s') \max_a Q(s,a)
$$

Quality functionと呼ぶ

もうひとつ考えようとおもうと、

$$
C(s,a) = \gamma \sum_{s'} T(s,a,s')\max_{a'} (R(s',a')+C(s',a'))
$$

これをContinuation functionとよぶ。すべてをBellman equationとよぶ。

$$
V(s) = \max_a(Q(s,a)) = \max_a(R(s,a)+C(s,a))
$$

$$
Q(s,a) = R(s,a) +\gamma\sum_{s'} T(s,a,s')V(s') = R(s,a) + C(s,a)
$$

$$
C(s,a) = \gamma \sum_{s'} T(s,a,s')V(s') = \gamma \sum_{s'} (T(s,a,s')\max_{a'}Q(s',a'))
$$

Cはその状態からそのアクションをとったときの未来のValueの期待値

Vはその状態で一番いいアクションをとったときの...

数学的には$R(s,a)$は$R(s)$と別の状態を考えれば書けるんだけど、この差が重要になってくる。

## Lesson 3 : Temporal difference learning

In [ ]:
環境が分からないのがreinforcement learningとMDPの違い



* plan : fixed sequence of actions
* conditional plan : include "if" but fixed
* stationary policy : mapping from state to action→large

policyの評価:そのpolicyでうまれるutilityの期待値


In [2]:
0.8**2 + 0.8**4 + 0.8**2 + (-0.2)*0.8

0.6400000000000001

### TD(1) learning
エピソード$T$に対して、すべての状態$s$のeligibility$e(s)=0$、value関数$V_T(s)=V_{T-1}(s)$で初期化する。

1ステップ$s_{t-1}\rightarrow^{r_t}s_t$動いたら  
eligibilityを更新 $e(s_{t-1})\leftarrow e(s_{t-1})+1$  
すべての状態$s$について  

Value関数の更新:
$$V_T(s) \leftarrow V_T(s) + \alpha_T (r_t + \gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1}))e(s)$$
eligibilityの更新:
$e(s) \leftarrow \gamma e(s)$

例として$s_1\rightarrow^{r_2} s_2 \rightarrow^{r_3} s_3 \rightarrow^{r_4} s_4\cdots  \rightarrow^{r_n} s_n $を考える。各ステップでの$s_1$についてのvalue関数は

$$
\begin{aligned}
\Delta V_T(s_1)(1) &= \alpha_T (r_2 + \gamma V_{T-1}(s_2) - V_{T-1} (s_1))  \gamma^0 \\
\Delta V_T(s_1)(2) &= \alpha_T (r_3 + \gamma V_{T-1}(s_3) - V_{T-1} (s_2))  \gamma^1 \\
\Delta V_T(s_1)(3) &= \alpha_T (r_4 + \gamma V_{T-1}(s_4) - V_{T-1} (s_3)) \gamma^2 \\
&\vdots \\
\Delta V_T(s_1)(n-1) &= \alpha_T (r_n + \gamma V_{T-1}(s_n) - V_{T-1} (s_{n-1})) \gamma^{n-2}
\end{aligned}
$$

全ステップを足すと、



$$
\Delta V_T(s_1) = \alpha_T(\sum_{i=0} r_{i+2} \gamma^i +\gamma^{n-1} V_{T-1}(s_n) - V_{T-1}(s_1))
$$

となって、未来の状態のvalue関数を割り引きながら足したものとの差分を更新することになる。

なんですべての状態のvalue関数についてある状態にうつったときのvalue関数を足すの？
→そのエピソードにおいてその状態にうつらなければ、$e(s)=0$だから、足されない。1ステップごとに$e(s)$に$\gamma$で割り引かれるので、未来のrewardも伝搬するようになっている。

TD(1) learningの問題点 : 更新に一回のエピソードしか使わないこと。全観測エピソードのなかで一回しか出てこなかった状態についてのバイアスがすごい大きいままになる

### TD(0) 
1ステップ動いたら、前にいた状態$s_{t-1}$についてだけ: 

$$
V_T(s_{t-1}) \leftarrow V_T(s_{t-1}) + \alpha_T (r_t + \gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1}))e(s) 
$$

こうすれば、ある状態についての更新が、偶然出た経路によって変にバイアスされない。これは最尤推定していることになる。

### TD(λ)


エピソード$T$に対して、すべての状態$s$のeligibility$e(s)=0$、value関数$V_T(s)=V_{T-1}(s)$で初期化する。

1ステップ$s_{t-1}\rightarrow^{r_t}s_t$動いたら  
eligibilityを更新 $e(s_{t-1})\leftarrow e(s_{t-1})+1$  
すべての状態$s$について  

Value関数の更新:
$$V_T(s) \leftarrow V_T(s) + \alpha_T (r_t + \gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1}))e(s)$$
eligibilityの更新:
$e(s) \leftarrow \lambda\gamma e(s)$

eligibilityの更新を$\lambda$割り引くだけ

TD(λ)は何をしているか？
k-step estimaterを次にように考える

あるエピソード$T$での更新として

$$
\begin{aligned}
1& : V_T(s_t) \leftarrow  V_T(s_t)  + \alpha_T (r_{t+1} + \gamma V_{T-1} (s_{t+1})  - V_{T-1}(s_t)) \\
2 &: V_T(s_t) \leftarrow  V_T(s_t)  + \alpha_T (r_{t+1} + \gamma V_{T-1} (s_{t+1}) + \gamma^2 V_{T-1} (s_{t+2}) - V_{T-1}(s_t)) \\
& \vdots \\
\infty&  : V_T(s_t) \leftarrow  V_T(s_t)  + \alpha_T (r_{t+1} + \gamma V_{T-1} (s_{t+1}) + \gamma^2 V_{T-1} (s_{t+2})  + \cdots - V_{T-1}(s_t)) \\
\end{aligned}
$$

1　step がTD(0)、$\infty$ステップがTD(1)に対応している。それぞれのestimaterに対して
$$
\begin{aligned}
1& :(1-\lambda)\\
2 &: \lambda(1-\lambda)\\
& \vdots \\
\infty& : \lambda^\infty\\
\end{aligned}
$$

を重みづけして考えていることになるらしい。たしかに、どんどん$\lambda$で落ちていくのでそうなってそう。